#### 1. MySQL 中 varchar 与 char 的区别？varchar(50) 中的 50 代表的涵义？
* varchar 与 char 的区别，char 是一种固定长度的类型，varchar 则是一种可变长度的类型。
* varchar(50) 中 50 的涵义最多存放 50 个字符。varchar(50) 和 (200) 存储 hello 所占空间一样，但后者在执行`order by`语法时消耗更多内存, 因为order by按照采用最大的字节数(fixed_length)计算.
 ```bash
 所以，实际场景下，选择合适的 varchar 长度还是有必要的。
 ```
 
#### 2.  int(11) 中的 11 代表什么涵义？
* int(n) 中的 n ，不影响字段存储的范围, 无论n取何值, 都是4字节存储
* n值只表示显示宽度, 规定用于显示宽度小于指定的列宽度的值时从左侧填满0。eg: int(6)的字段显示数字11->"000011"

#### 3. 一张表，里面有 ID 自增主键，当 insert 了 17 条记录之后，删除了第 15,16,17 条记录，再把 MySQL 重启，再 insert 一条记录，这条记录的 ID 是 18 还是 15？
* 一般情况下，我们创建的表的类型是 InnoDB ，如果新增一条记录（不重启 MySQL 的情况下），这条记录的 ID 是18 ；但是如果重启 MySQL 的话，这条记录的 ID 是 15 。因为 InnoDB 表只把自增主键的最大 ID 记录到内存中，所以重启数据库或者对表 OPTIMIZE 操作，都会使最大 ID 丢失。
* 但是，如果我们使用表的类型是 MyISAM ，那么这条记录的 ID 就是 18 。因为 MyISAM 表会把自增主键的最大 ID 记录到数据文件里面，重启 MYSQL 后，自增主键的最大 ID 也不会丢失。  

最后，还可以跟面试官装个 x ，生产数据，不建议进行物理删除记录。

#### 4. 为什么mysql的默认引擎是innodb
主要原因如下：
* 【最重要】支持事务。
* 支持锁行和锁表，能支持更多的并发量。
* 查询不加锁，完全不影响查询。
* 支持崩溃后恢复。

#### 5. 为什么mysql的count()使用innodb时比使用MyISAM时慢
对于 SELECT COUNT(*) FROM table 语句，在没有 WHERE 条件的情况下，InnoDB 比 MyISAM 可能会慢很多，尤其在大表的情况下。因为，InnoDB 是去实时统计结果，会全表扫描；而 MyISAM 内部维持了一个计数器，预存了结果，所以直接返回即可。
https://blog.csdn.net/qq_15037231/article/details/81179383

### 6. 【重点】什么是索引？
mysql的索引, 记录了数据所在文件的行数, 根据这个行数可以直接取到数据, 不用扫表
* **索引有什么好处？**
    1. 降低IO成本: 因为索引可以直接定位到行, 避免了大块的数据移动从磁盘到内存
    2. 降低数据排序的成本: 索引的建立过程中会对数据排序(B+树的叶子结点形成成了一个链表,从左到右值依次增大); 若该字段正好需要排序，则正好降低了排序的成本。 
    
* **索引有什么坏处？**
    1. 索引使得update操作变慢, 因为更新数据时也要更新数据的索引表
    
* **索引的使用场景？**
    1. 对非常小的表，大部分情况下全表扫描效率更高。
    2. 对中大型表，索引非常有效。
    3. 特大型的表，建立和使用索引的代价不断增长; 此时可以选择"分库分表"
      
* **索引的类型？** 
    1. 普通索引：最基本的索引，没有任何约束。
    2. 唯一索引：与普通索引类似，但具有唯一性约束。
    3. 主键索引：特殊的唯一索引，不允许有空值。
    4. 复合索引：将多个列组合在一起创建索引，可以覆盖多个列。
    5. 外键索引：只有InnoDB类型的表才可以使用外键索引，保证数据的一致性、完整性和实现级联操作。
    6. 全文索引：为了模糊查询而设计的整个字段的索引; MySQL 自带的全文索引只能用于 InnoDB、MyISAM ，并且只能对英文进行全文检索，一般使用全文索引引擎。
    ```
    在一般情况下，模糊查询都是通过 like 的方式进行查询。但是，对于海量数据，这并不是一个好办法，在 like “value%” 可以使用索引，但是对于 like “%value%” 这样的方式，执行全表查询，这在数据量小的表，不存在性能问题，但是对于海量数据，全表扫描是非常可怕的事情,所以 like 进行模糊匹配性能很差。常用的全文索引引擎的解决方案有 Elasticsearch、Solr 等等。最为常用的是 Elasticsearch 。

    ```
     
* **索引的“创建”原则有哪些** 
    1. 最适合索引的列是出现在 WHERE 子句中的列，或连接子句中的列，而不是出现在 SELECT 关键字后的列。
    2. 索引列的基数越大(索引字段取不同值的个数越大)，索引效果越好。
    3. 根据情况创建复合索引，复合索引可以提高查询效率。
        ```
        因为复合索引的基数会更大。
        ```
    4. 避免创建过多的索引，索引会额外占用磁盘空间，降低写操作效率。
    5. 主键尽可能选择较短的数据类型，可以有效减少索引的磁盘占用提高查询效率。
    6. 对字符串进行索引，应该定制一个前缀长度，可以节省大量的索引空间。
    
* **MySQL 索引的“使用”注意事项？**<span id="id1"></span>
    1. 应尽量避免在 `WHERE` 子句中使用 `!= 或 <>` 操作符，否则将引擎放弃使用索引而进行全表扫描。优化器将无法通过索引来确定将要命中的行数,因此需要搜索该表的所有行。
    ```
    注意，column IS NULL 也是不可以使用索引的。
    ```
    2. 应尽量避免在 WHERE 子句中使用 OR 来连接条件，否则将导致引擎放弃使用索引而进行全表扫描，
    ```
    如：SELECT id FROM t WHERE num = 10 OR num = 20 。
    ```
    3. 应尽量避免在 WHERE 子句中对字段进行表达式操作，这将导致引擎放弃使用索引而进行全表扫描。
    ```
    select * from news where id / 100 = 1
    为了使用索引，防止执行全表扫描，可以进行改造。
    select * from news where id = 1 * 100
    ```
    4. 应尽量避免在 WHERE 子句中对字段进行函数操作，这将导致引擎放弃使用索引而进行全表扫描。
    ```
    select * from news where year(publish_time) < 2017
    为了使用索引，防止执行全表扫描，可以进行改造。
    select * from news where publish_time < '2017-01-01'
    ```
    6. 多个单列索引并不是最佳选择, 应该使用复合索引, 复合索引遵循前缀原则。
    ```
    MySQL 只能使用一个索引，会从多个索引中选择一个限制最为严格的索引，因此，为多个列创建单列索引，并不能提高 MySQL 的查询性能。
    假设，有两个单列索引，分别为 news_year_idx(news_year) 和 news_month_idx(news_month)。现在，有一个场景需要针对资讯的年份和月份进行查询，那么，SQL 语句可以写成：
    select * from news where news_year = 2017 and news_month = 1
    事实上，MySQL 只能使用一个单列索引。为了提高性能，可以使用复合索引 news_year_month_idx(news_year, news_month) 保证 news_year 和 news_month 两个列都被索引覆盖。
    ```
    6. 复合索引的最左前缀原则(只有在where条件中使用了复合索引的第一个字段，索引才会被使用)
    ```
    复合索引遵守“最左前缀”原则，即在查询条件中使用了复合索引的第一个字段，索引才会被使用。因此，在复合索引中索引列的顺序至关重要。如果不是按照索引的最左列开始查找，则无法使用索引。
    假设，有一个场景只需要针对资讯的月份进行查询，那么，SQL 语句可以写成：
    select * from news where news_month = 1
    此时，无法使用 news_year_month_idx(news_year, news_month) 索引，因为遵守“最左前缀”原则，在查询条件中没有使用复合索引的第一个字段，索引是不会被使用的。
    ```
    7. 如果 MySQL 评估使用索引比全表扫描更慢，会放弃使用索引。如果此时想要索引，可以在语句中添加强制索引。
    8. 列类型是字符串类型，查询时一定要给值加引号，否则索引失效。
    9. LIKE 查询可能会引起索引失效:    
       like 的方式进行查询，在 like “value%” 可以使用索引，但是对于 like “%value%” 这样的方式，执行全表查询。如果需要模糊匹配，可以使用全文索引。
         
* **以下三条 SQL 如何建索引，只建一条怎么建？**
```sql
WHERE a = 1 AND b = 1
WHERE b = 1
WHERE b = 1 ORDER BY time DESC
```
按照复合索引的最左前缀原则, 以顺序 b , a, time 建立复合索引，`CREATE INDEX table1_b_a_time ON index_test01(b, a, time)。`对于第一条 SQL ，因为最新 MySQL 版本会优化 WHERE 子句后面的列顺序，以匹配复合索引顺序。

* **一个查询用到了哪个索引，如何查看?**   
`EXPLAIN` 显示了 MYSQL 如何使用索引来处理 `SELECT` 语句以及连接表,可以帮助选择更好的索引和写出更优化的查询语句。  
使用方法，在 SELECT 语句前加上 EXPLAIN 就可以了。

### 7. 【重点】MySQL 索引的原理？
* **innoDB使用B+树作为索引存储的数据结构**
    1. innodb中, 有`页`的概念, 默认页的大小设置为16kb, 表示磁盘和内存IO时一次的交换大小. 由于B+树数据结构中, 处于外存的每个文件就是B+树的一个节点, 因此, Innodb一个索引节点最大有16kb,按照INT栈4字节, 指针类型占4字节计算, 可以存放(16kb/4B+4B)=1000个索引. 因此如果把inodb的索引看成1000阶的B+树, 则3层的B+树就可对$10^3 * 10^3 * 10^3=10亿$条记录做索引
    2. innoDB索引的`根节点会常驻内存`, 只有在切换节点或修改节点时才发生磁盘IO, 且由于innodb索引是高阶B+树, 因此, IO次数一般3次左右
    3. 对于一般的B+树, 只有叶子结点才存储真正的索引, 其它节点只是存储的key序列, innodB的索引文件也一样, 只有叶子结点才记录着被索引的row在磁盘什么位置. 但与一般B+树不同的是, 一般B+树只会把叶子结点从左到右串成一个链表来应付顺序查找, 而innodb将所有节点串成环形链表, 且维护2个指针:一个指向根节点,一个指向key最小的叶子结点; 以此来应付对于主键的`范围查找和分页查找`
     
* **有哪些索引类型: 聚簇索引和非聚簇索引**   
索引分为聚簇索引主键索引和非主键索引, 聚簇索引的叶子结点<key,value>是<主键值, row的数据>; 非聚簇索引的叶子结点<key.value>是<非主键列的值, 主键值>; 非聚簇索引是聚簇索引的二级索引;
    1. 聚簇索引和非聚簇索引   
        1. 在《数据库原理》一书中说, 聚簇索引的叶子结点就是数据节点; 非聚簇索引的叶子结点仍然是索引节点, 只不过有能指向数据的指针.  
        2. innoDB中, 用聚簇索引对主键做索引, 用非聚簇索引索引做非主键列的索引;   
           非主键列的索引其叶子结点存储着其对应行的主键值. 在非主键列上查找时, 先通过非聚簇索引找到属于自己的主键值, 然后用这个主键值在聚簇索引中查找row的数据. 因此, 可以说非聚簇索引是聚簇索引的二级索引. 之所以非聚簇索引存储的是主键值而不是指向数据行的行指针, 是为了避免当数据移动时维护B+树的结构需要同时修改锁哥索引文件, 有了这种设计, 值修改主键索引(聚簇索引)的row index即可
    2. 没设置主键的情况下, innoDB如何安排聚簇索引  
      如果没设置主键, innoDB会隐式的将唯一且非空的列作为主键, 如果没有这样的列, 生成一个隐藏列作为主键
   
* **聚簇索引有什么应该注意的地方**  
  聚簇索引最大限度的提高了IO密集型应用的性能, 但也有如下几个限制  
    1. 聚簇索引的插入速度严重依赖于插入顺序  
       顺序插入是最快的插入方式, 因为创建B+树的节点时. 顺序插入的索引都会处于一个页中, 非顺序插入可能会插入到不同页, 增大了IO次数. 这也是**为什么会定义一个与业务无关的自增主键作为聚簇索引**的原因  
    2. 主键一般要定义为不可更新  
       因为更新主键, 可能会导致其所在行移动, 行指针变化, 所以我们一般定义主键为不可更新。
    3. 二级索引(非聚簇索引)访问需要两次索引查找，第一次找到主键值，第二次根据主键值(聚簇索引)找到行数据。
    4. 主键 ID 建议使用整型。  
       因为，每个主键索引的 B+Tree 节点的键值可以存储更多主键 ID ，每个非主键索引的 B+Tree 节点的数据可以存储更多主键 ID 。
  
* **什么是最左匹配原则**   
使用复合索引时, B+树中的key也是符合项, 比如索引`(name, age, sex)` 的时候，B+Tree 是按照从左到右的顺序来建立搜索树的。先用待查记录的name比较, 找到name相同的索引后再比较age, 最后比较sex. 如果查询语句中的条件只有(age,sex), 则因为缺少最左面的name条件, 导致索引全部不可用. 这就是最左匹配原则  
    * 比如当 (张三, 20, F) 这样的数据来检索的时候，B+Tree 会优先比较 name 来确定下一步的所搜方向，如果 name 相同再依次比较 age 和 sex ，最后得到检索的数据。
    * 但当 (20, F) 这样的没有 name 的数据来的时候，B+Tree 就不知道下一步该查哪个节点，因为建立搜索树的时候 name 就是第一个比较因子，必须要先根据 name 来搜索才能知道下一步去哪里查询。
    * 比如当 (张三, F) 这样的数据来检索时，B+Tree 可以用 name 来指定搜索方向，但下一个字段 age 的缺失，所以只能把名字等于张三的数据都找到，然后再匹配性别是 F 的数据了。

### 8. 【重点】请说说 MySQL 的四种事务隔离级别？

### 9. Innodb 的事务与日志的实现方式
https://dev.mysql.com/doc/refman/5.7/en/innodb-locking-transaction-model.html

### 10. 【重要】MySQL 查询执行顺序？
1. RBO谓词下推, 限制性where
2. join
3. group by
4. having
5. order by
6. limit 

### 11. 【重要】聊聊 MySQL SQL 优化？
1. 如果使用了索引, 索引字段的where条件要满足[MySQL 索引的“使用”注意事项？](#id1)
2. 用exist代替in, 或用join代替in
3. 尽量使用数字类型的字段  
   若只含数值信息的字段尽量不要设计为字符型，这会降低查询和连接的性能，并会增加存储开销。这是因为引擎在处理查询和连接时会逐个比较字符串中每一个字符，而对于数字型而言只需要比较一次就够了。

###  12.【加分】什么是 MVCC ？

### 13. MySQL binlog 的几种日志录入格式以及区别
https://dev.mysql.com/doc/refman/5.7/en/binary-log.html